In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn import metrics
from collections import defaultdict
import matplotlib.pyplot as plt
import glob


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics.cluster import contingency_matrix, homogeneity_score, completeness_score, adjusted_rand_score, adjusted_mutual_info_score, v_measure_score
from sklearn.metrics.cluster import contingency_matrix

## Import libraries
import re
import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords' )

import string
from string import punctuation
import matplotlib.pyplot as plt

In [2]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
import random
import pandas as pd
import numpy as np
import re, json, datetime, pytz
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from collections import Counter
import nltk

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [8]:
import pytz
import datetime

pst_tz = pytz.timezone('America/Los_Angeles')

In [3]:
from happytransformer import HappyTextClassification
happy_tc = HappyTextClassification(model_type="DISTILBERT", 
                                       model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

03/13/2022 03:39:25 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [4]:
from transformers import pipeline
pipe = pipeline(task='text2text-generation', model='facebook/m2m100_418M')

In [5]:
def transformer_sentiment(tweet):
    
    result = happy_tc.classify_text(tweet)
    if result.label == 'NEGATIVE':
        return -1, -result.score
    else:
        return 1, result.score
    

In [6]:
def translate_and_transformer_sentiment(tweet):
    
    tweet = pipe(tweet, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='en'))
    tweet = tweet[0]['generated_text']
    result = happy_tc.classify_text(tweet)
    if result.label == 'NEGATIVE':
        return -1, -result.score
    else:
        return 1, result.score

In [7]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [9]:
def get_data(text):
    with open(text+'.txt', 'r',encoding="utf8") as f:
        retweet_count, fav_count, followers, place, title, time = [],[],[],[],[],[]
        sentiment_polarity, sentiment_score = [], []
        pst_tz = pytz.timezone('US/Pacific') 
        for line in f:
            tweet = json.loads(line)
            title.append(tweet['title'])
            retweet_count.append(tweet['metrics']['citations']['total']) 
            fav_count.append(tweet['tweet']['favorite_count'])
            followers.append(tweet['author']['followers'])
            time.append(datetime.datetime.fromtimestamp(tweet['citation_date'], pst_tz))
            place.append(tweet['tweet']['user']['location'])
            cleaned_tweet = clean_tweet(tweet['title'])
            if not tweet['tweet']['lang'] == 'en':
                senti_polarity, senti_score = translate_and_transformer_sentiment(cleaned_tweet)
                sentiment_polarity.append(senti_polarity)
                sentiment_score.append(senti_score)
            else:
                senti_polarity, senti_score = transformer_sentiment(cleaned_tweet)
                sentiment_polarity.append(senti_polarity)
                sentiment_score.append(senti_score)
            break
    d = {'title':title, 'retweet_count':retweet_count, 'favourite_count': fav_count, 'followers':followers,
         'time':time,'place':place, 'sentiment_polarity':sentiment_polarity, 'sentiment_score':sentiment_score}
    df = pd.DataFrame(d)
    return df   